# Bird Co-occurences
### Analyzing Indian ebird data of co-occurences for habitats and geography

We analyze the e-bird data for _co-occurences_ of species, i.e., species occuring together more than expected by their separate frequencies. We can formulate this as the following question

### Question: What causes bird species to occur together?

We shall see that such _co-occurences_ occur for two reasons - geography and habitat.

With more data, one can reverse this analysis and divide India into geographical regions and habitats, from a _bird's eye view_, based on co-occurences. Clearly the present e-bird data is inadequate for this as we only see some regions and habitats in it.

**Code:** This is a jupyter notebook written in scala.

## Setup and loading data

We first add all our dependencies, both our own code and other stuff on which it depends. The code here is  on the repository for some of our other code at [Proving-Ground](https://github.com/siddhartha-gadgil/ProvingGround)

In [1]:
classpath.addPath("/home/gadgil/code/ProvingGround/deepwalk/target/scala-2.11/deepwalk4s_2.11-0.8.jar")

In [2]:
classpath.add("com.lihaoyi" %% "ammonite-ops" % "0.7.7")


1 new artifact(s)


1 new artifacts in macro
1 new artifacts in runtime
1 new artifacts in compile


In [3]:
import ammonite.ops._

import ammonite.ops._

The e-bird data has been pre-processed (not in this notebook) and saved in various files. 
We first read how many checklists contain a given bird, and how many contain both birds in a pair.

In [4]:
val data = pwd / up / 'data

data: Path = /home/gadgil/code/ProvingGround/data

In [5]:
val freqF = data / "frequencies.tsv"
val freqs = read.lines(freqF) map (_.split("\t")) map {case Array(a, n) => (a, n.toInt)} sortBy((an) => - an._2)

freqF: data.ThisType = /home/gadgil/code/ProvingGround/data/frequencies.tsv
freqs: Vector[(String, Int)] = Vector(
  ("Corvus splendens", 12538),
  ("Acridotheres tristis", 12509),
  ("Halcyon smyrnensis", 11176),
  ("Corvus macrorhynchos", 10495),
  ("Dicrurus macrocercus", 9785),
  ("Ardeola grayii", 9660),
  ("Pycnonotus cafer", 9551),
  ("Pycnonotus jocosus", 8892),
  ("Streptopelia chinensis", 8817),
  ("Orthotomus sutorius", 8780),
  ("Copsychus saularis", 8545),
  ("Centropus sinensis", 8394),
  ("Psilopogon viridis", 8126),
  ("Psittacula krameri", 7858),
  ("Columba livia", 7658),
  ("Milvus migrans", 7596),
  ("Leptocoma zeylonica", 7447),
  ("Microcarbo niger", 7419),
  ("Eudynamys scolopaceus", 7410),
...

In [6]:
val coF = data /"co-occurences.tsv"
def pairs = read.lines(coF) map (_.split("\t")) map {case Array(a, b, n) => (a, b, n.toInt)}

coF: data.ThisType = /home/gadgil/code/ProvingGround/data/co-occurences.tsv
defined function pairs

We use scientific names for analysis, but we also have extracted common names since they are nicer to see.

In [7]:
val commonF = data / "common-names.tsv"
val commonNames = read.lines(commonF) map (_.split("\t")) map {case Array(sc, comm) => (sc, comm)} toMap

commonF: data.ThisType = /home/gadgil/code/ProvingGround/data/common-names.tsv
commonNames: Map[String, String] = Map(
  "Clamator coromandus" -> "Chestnut-winged Cuckoo",
  "Alauda arvensis/gulgula" -> "Eurasian/Oriental Skylark",
  "Ardenna carneipes" -> "Flesh-footed Shearwater",
  "Ichthyophaga ichthyaetus" -> "Gray-headed Fish-Eagle",
  "Otus lettia" -> "Collared Scops-Owl",
  "Aythya nyroca" -> "Ferruginous Duck",
  "Aegithalos iouschistos" -> "Black-browed Tit",
  "Anatidae sp." -> "waterfowl sp.",
  "Cinclidium leucurum" -> "White-tailed Robin",
  "Xenus cinereus" -> "Terek Sandpiper",
  "Horornis flavolivaceus" -> "Aberrant Bush-Warbler",
  "Catreus wallichii" -> "Cheer Pheasant",
  "Accipiter butleri" -> "Nicobar Sparrowhawk",
  "Merops apiaster" -> "European Bee-eater",
  "Psittacula eupatria" -> "Alexandrine Parakeet",
  "Hydroprogne caspia" -> "Caspian Tern",
  "Elachura formosa" -> "Spotted Elachura",
  "Corvus macrorhynchos" -> "Large-billed Crow",
  "Dryocopus javensis"

Let us look at the 250 most common birds, where how common is measured by how many checklists contain a bird.

In [8]:
val top250 = freqs.take(250).map(_._1)
show((top250 map (commonNames)).zipWithIndex)

Vector(
  ("House Crow", 0),
  ("Common Myna", 1),
  ("White-throated Kingfisher", 2),
  ("Large-billed Crow", 3),
  ("Black Drongo", 4),
  ("Indian Pond-Heron", 5),
  ("Red-vented Bulbul", 6),
  ("Red-whiskered Bulbul", 7),
  ("Spotted Dove", 8),
  ("Common Tailorbird", 9),
  ("Oriental Magpie-Robin", 10),
  ("Greater Coucal", 11),
  ("White-cheeked Barbet", 12),
  ("Rose-ringed Parakeet", 13),
  ("Rock Pigeon", 14),
  ("Black Kite", 15),
  ("Purple-rumped Sunbird", 16),
  ("Little Cormorant", 17),
  ("Asian Koel", 18),
  ("Cattle Egret", 19),
  ("Red-wattled Lapwing", 20),
  ("Brahminy Kite", 21),
  ("Rufous Treepie", 22),
  ("Green Bee-eater", 23),
  ("Little Egret", 24),
  ("Ashy Prinia", 25),
  ("Purple Sunbird", 26),
  ("Jungle Babbler", 27),
  ("Black-rumped Flameback", 28),
  ("Jungle Myna", 29),
  ("Barn Swallow", 30),
  ("Coppersmith Barbet", 31),
  ("Pale-billed Flowerpecker", 32),
  ("Indian Peafowl", 33),
  ("Intermediate Egret", 34),
  ("White-breasted Waterhen", 35),
  (

top250: Vector[String] = Vector(
  "Corvus splendens",
  "Acridotheres tristis",
  "Halcyon smyrnensis",
  "Corvus macrorhynchos",
  "Dicrurus macrocercus",
  "Ardeola grayii",
  "Pycnonotus cafer",
  "Pycnonotus jocosus",
  "Streptopelia chinensis",
  "Orthotomus sutorius",
  "Copsychus saularis",
  "Centropus sinensis",
  "Psilopogon viridis",
  "Psittacula krameri",
  "Columba livia",
  "Milvus migrans",
  "Leptocoma zeylonica",
  "Microcarbo niger",
  "Eudynamys scolopaceus",
...

We shall analyse which birds are seen together, but focussing attention on pairs with both among the top 250. This is because when we map birds to vectors, if we include all birds then the common ones cluster together.

In [9]:
val topSet = top250.toSet
val scientificNames = (commonNames map {case (s, c) => (c, s)}).toMap
val bothSeen = {for {(a, b, n) <- pairs if topSet.contains(a) && topSet.contains(b)} yield((a, b), n)}.toMap

topSet: Set[String] = Set(
  "Psittacula eupatria",
  "Corvus macrorhynchos",
  "Ardea alba",
  "Mycteria leucocephala",
  "Zosterops palpebrosus",
  "Dendrocitta leucogastra",
  "Ploceus philippinus",
  "Artamus fuscus",
  "Cyornis tickelliae",
  "Platalea leucorodia",
  "Dicaeum agile",
  "Picus chlorolophus",
  "Eumyias thalassinus",
  "Dendrocopos nanus",
  "Acridotheres ginginianus",
  "Circus aeruginosus",
  "Merops leschenaulti",
  "Phaenicophaeus viridirostris",
  "Ficedula parva",
...
scientificNames: Map[String, String] = Map(
  "roller sp." -> "Coracias sp.",
  "Whimbrel" -> "Numenius phaeopus",
  "Laughing Dove" -> "Streptopelia senegalensis",
  "Tawny-breasted Wren-Babbler" -> "Spelaeornis longicaudatus",
  "godwit sp." -> "Limosa sp.",
  "White-breasted Waterhen" -> "Amaurornis phoenicurus",
  "Eurasian Wren" -> "Troglodytes troglodytes",
  "scops-owl sp." -> "Otus sp.",
  "Indian Cuckoo" -> "Cuculus micropterus",
  "Siberian Blue Robin" -> "Larvivora cyane",
  "Velvet-fr

In [10]:
val p = freqs.toMap

p: Map[String, Int] = Map(
  "Clamator coromandus" -> 42,
  "Alauda arvensis/gulgula" -> 2,
  "Ardenna carneipes" -> 49,
  "Ichthyophaga ichthyaetus" -> 48,
  "Otus lettia" -> 27,
  "Aythya nyroca" -> 188,
  "Aegithalos iouschistos" -> 14,
  "Anatidae sp." -> 55,
  "Cinclidium leucurum" -> 31,
  "Xenus cinereus" -> 221,
  "Horornis flavolivaceus" -> 37,
  "Catreus wallichii" -> 11,
  "Accipiter butleri" -> 1,
  "Merops apiaster" -> 17,
  "Psittacula eupatria" -> 527,
  "Hydroprogne caspia" -> 190,
  "Elachura formosa" -> 20,
  "Corvus macrorhynchos" -> 10495,
  "Dryocopus javensis" -> 174,
...

The _co-occurence_ of two species is the ratio of the probability that they are seen together to what this probability would be if they were independent. We don't actually take the ratio but a constant multiple of the ratio as this makes no difference in the analysis.

In [11]:
def coOccurence(a: String, b: String) = 10000.0 * bothSeen((a, b)) / (p(a) * p(b))

defined function coOccurence

We already can see some interesting patterns from the data. Below we see the species that co-occur the most (first by scientific name, then the top 1000 pairs by common name)

In [12]:
val topPairs = for (a <- top250; b <- top250 if a != b) yield (a, b)


topPairs: Vector[(String, String)] = Vector(
  ("Corvus splendens", "Acridotheres tristis"),
  ("Corvus splendens", "Halcyon smyrnensis"),
  ("Corvus splendens", "Corvus macrorhynchos"),
  ("Corvus splendens", "Dicrurus macrocercus"),
  ("Corvus splendens", "Ardeola grayii"),
  ("Corvus splendens", "Pycnonotus cafer"),
  ("Corvus splendens", "Pycnonotus jocosus"),
  ("Corvus splendens", "Streptopelia chinensis"),
  ("Corvus splendens", "Orthotomus sutorius"),
  ("Corvus splendens", "Copsychus saularis"),
  ("Corvus splendens", "Centropus sinensis"),
  ("Corvus splendens", "Psilopogon viridis"),
  ("Corvus splendens", "Psittacula krameri"),
  ("Corvus splendens", "Columba livia"),
  ("Corvus splendens", "Milvus migrans"),
  ("Corvus splendens", "Leptocoma zeylonica"),
  ("Corvus splendens", "Microcarbo niger"),
  ("Corvus splendens", "Eudynamys scolopaceus"),
  ("Corvus splendens", "Bubulcus ibis"),
...

In [13]:
val together = topPairs.sortBy((ab) => - coOccurence(ab._1, ab._2)).filter((ab) => (ab._1 < ab._2))

together: Vector[(String, String)] = Vector(
  ("Hypsipetes leucocephalus", "Psilopogon virens"),
  ("Charadrius alexandrinus", "Charadrius mongolus"),
  ("Anas crecca", "Anas strepera"),
  ("Anas strepera", "Anser indicus"),
  ("Anser indicus", "Tadorna ferruginea"),
  ("Calidris minuta", "Calidris temminckii"),
  ("Phylloscopus xanthoschistos", "Saxicola ferreus"),
  ("Myophonus caeruleus", "Phylloscopus xanthoschistos"),
  ("Pycnonotus leucogenys", "Saxicola ferreus"),
  ("Phoenicurus ochruros", "Sylvia curruca"),
  ("Phylloscopus xanthoschistos", "Pycnonotus leucogenys"),
  ("Anas clypeata", "Anas strepera"),
  ("Myophonus caeruleus", "Psilopogon virens"),
  ("Hypsipetes leucocephalus", "Phylloscopus xanthoschistos"),
  ("Anas clypeata", "Anas crecca"),
  ("Anas strepera", "Tadorna ferruginea"),
  ("Ficedula parva", "Phylloscopus humei"),
  ("Hypsipetes leucocephalus", "Myophonus caeruleus"),
  ("Myophonus caeruleus", "Saxicola ferreus"),
...

In [14]:
show(together map {case (x, y) => (commonNames(x), commonNames(y))} take (1000))

Vector(
  ("Black Bulbul", "Great Barbet"),
  ("Kentish Plover", "Lesser Sand-Plover"),
  ("Green-winged Teal", "Gadwall"),
  ("Gadwall", "Bar-headed Goose"),
  ("Bar-headed Goose", "Ruddy Shelduck"),
  ("Little Stint", "Temminck's Stint"),
  ("Gray-hooded Warbler", "Gray Bushchat"),
  ("Blue Whistling-Thrush", "Gray-hooded Warbler"),
  ("Himalayan Bulbul", "Gray Bushchat"),
  ("Black Redstart", "Lesser Whitethroat"),
  ("Gray-hooded Warbler", "Himalayan Bulbul"),
  ("Northern Shoveler", "Gadwall"),
  ("Blue Whistling-Thrush", "Great Barbet"),
  ("Black Bulbul", "Gray-hooded Warbler"),
  ("Northern Shoveler", "Green-winged Teal"),
  ("Gadwall", "Ruddy Shelduck"),
  ("Red-breasted Flycatcher", "Hume's Warbler"),
  ("Black Bulbul", "Blue Whistling-Thrush"),
  ("Blue Whistling-Thrush", "Gray Bushchat"),
  ("Temminck's Stint", "Common Snipe"),
  ("Gray-hooded Warbler", "Great Barbet"),
  ("Gray-hooded Warbler", "White-throated Fantail"),
  ("Blue Whistling-Thrush", "Himalayan Bulbul"),
  (

We see that there are two kinds of pairs above:

* Those with the same geography, particularly those confined mainly to the himalayas or the western ghat.
* Those with similar habitats, most obviously water birds. Even better, we see waders co-occur with other waders and swimmers with other swimmers, and grassland birds occur with others.

## Force-directed layout

We visualize the results using a _force-directed layout_, using the implementation in _https://github.com/rsimon/scala-force-layout_

In [15]:
classpath.addPath("/home/gadgil/code/scala-force-layout/target/scala-2.11/scala-force-layout_2.11-0.4.0.jar")

In [16]:
import at.ait.dme.forcelayout._

import at.ait.dme.forcelayout._

In [17]:
import deepwalk4s._

import SvgGraphs._


import deepwalk4s._
import SvgGraphs._

In [18]:
val birdNodes = topSet.toVector map ((s) => Node(s, commonNames(s)))

birdNodes: Vector[Node] = Vector(
  Node(
    "Psittacula eupatria",
    "Alexandrine Parakeet",
    1.0,
    0,
    List(),
    List(),
    NodeState(
      Vector2D(0.45907578782237324, 0.019321055374912244),
      Vector2D(0.0, 0.0),
      Vector2D(0.0, 0.0)
    )
  ),
  Node(
    "Corvus macrorhynchos",
    "Large-billed Crow",
    1.0,
    0,
    List(),
...

In [19]:
val birdEdges = for ((x, y) <- topPairs) yield Edge(Node(x, commonNames(x)), Node(y, commonNames(y)), coOccurence(x, y))

birdEdges: Vector[Edge] = Vector(
  Edge(
    Node(
      "Corvus splendens",
      "House Crow",
      1.0,
      0,
      List(),
      List(),
      NodeState(
        Vector2D(-0.2187561722736484, 0.280245108196452),
        Vector2D(0.0, 0.0),
        Vector2D(0.0, 0.0)
      )
    ),
    Node(
      "Acridotheres tristis",
      "Common Myna",
      1.0,
      0,
...

In [20]:
val birdGraph = new SpringGraph(birdNodes, birdEdges)

birdGraph: SpringGraph = at.ait.dme.forcelayout.SpringGraph@106fad9c

In [21]:
birdGraph.doLayout(maxIterations = 3000)


In [22]:
val birdTriples = birdGraph.nodes.toVector map ((n) => (n.state.pos.x, n.state.pos.y, n.label))

birdTriples: Vector[(Double, Double, String)] = Vector(
  (244.18298343815485, 32.43422228152828, "Alexandrine Parakeet"),
  (46.99990366291408, -44.32769751148088, "Large-billed Crow"),
  (103.75698510233357, 21.28726034086808, "Great Egret"),
  (354.34239330408036, -15.263688749163816, "Painted Stork"),
  (-79.83362939002694, 81.0052870677415, "Oriental White-eye"),
  (-625.6861184459806, 22.044998806442837, "White-bellied Treepie"),
  (107.7530442402556, 59.83810036758661, "Baya Weaver"),
  (-116.02449762362188, -73.56173118989649, "Ashy Woodswallow"),
  (-101.5849214576269, -53.550942694970516, "Tickell's Blue-Flycatcher"),
  (387.5527995984451, -31.775783023892277, "Eurasian Spoonbill"),
  (-116.5965713804115, 84.52934985414319, "Thick-billed Flowerpecker"),
  (-252.60937581961576, 91.1790621222419, "Lesser Yellownape"),
  (-146.62444817326747, 94.68069989097435, "Verditer Flycatcher"),
  (-86.58665188188104, 65.21678974804064, "Brown-capped Woodpecker"),
  (642.5778062127378, 82.

In [23]:
val bigBirdPlot = scatterPlot(birdTriples, width = 1200, height = 500, r = 2)

bigBirdPlot: String = """

      <div>
      
    
    <style>
      .labl {
        display: none;
      }
      .labelled:hover + .labl {
        display: inline;
      }
    </style>
    

    <svg version="1.1"
   baseProfile="full"
   width="1200" height="500"
   xmlns="http://www.w3.org/2000/svg">

...

### Co-occurence plots

On hovering over points in the pictures below, we can see what bird they represent. One can wander around the picture to see how birds have clustered.

In [24]:
display.html(bigBirdPlot)

Alexandrine Parakeet 
 

 
 Large-billed Crow 
 

 
 Great Egret 
 

 
 Painted Stork 
 

 
 Oriental White-eye 
 

 
 White-bellied Treepie 
 

 
 Baya Weaver 
 

 
 Ashy Woodswallow 
 

 
 Tickell's Blue-Flycatcher 
 

 
 Eurasian Spoonbill 
 

 
 Thick-billed Flowerpecker 
 

 
 Lesser Yellownape 
 

 
 Verditer Flycatcher 
 

 
 Brown-capped Woodpecker 
 

 
 Bank Myna 
 

 
 Eurasian Marsh-Harrier 
 

 
 Chestnut-headed Bee-eater 
 

 
 Blue-faced Malkoha 
 

 
 Red-breasted Flycatcher 
 

 
 Pied Kingfisher 
 

 
 Oriental Turtle-Dove 
 

 
 Tricolored Munia 
 

 
 Malabar Gray Hornbill 
 

 
 Short-toed Snake-Eagle 
 

 
 Southern Hill Myna 
 

 
 Himalayan Bulbul 
 

 
 Common Iora 
 

 
 Greenish Warbler 
 

 
 Bay-backed Shrike 
 

 
 Greater Coucal 
 

 
 Gray Wagtail 
 

 
 Indian Gray Hornbill 
 

 
 Red Spurfowl 
 

 
 Stork-billed Kingfisher 
 

 
 Bronze-winged Jacana 
 

 
 Orange Minivet 
 

 
 Red-wattled Lapwing 
 

 
 Pale-billed Flowerpecker 
 

 
 Jungle Prinia 
 

 
 Eurasian Kestrel 
 

 
 Asian Openbill 
 

 
 Indian Golden Oriole 
 

 
 Red-whiskered Bulbul 
 

 
 White Wagtail 
 

 
 Booted Warbler 
 

 
 Kentish Plover 
 

 
 Gray-fronted Green-Pigeon 
 

 
 Jungle Owlet 
 

 
 Long-tailed Shrike 
 

 
 House Sparrow 
 

 
 white egret sp. 
 

 
 Rufous Woodpecker 
 

 
 Hair-crested Drongo 
 

 
 Malabar Woodshrike 
 

 
 Ashy Drongo 
 

 
 Plain Prinia 
 

 
 Indian Silverbill 
 

 
 Siberian Stonechat 
 

 
 Green/Greenish Warbler 
 

 
 Rufous Treepie 
 

 
 Brown-headed Barbet 
 

 
 Gray Junglefowl 
 

 
 Little Stint 
 

 
 Osprey 
 

 
 Pied Cuckoo 
 

 
 Little Ringed Plover 
 

 
 Garganey 
 

 
 Coppersmith Barbet 
 

 
 Black Bulbul 
 

 
 Spot-billed Pelican 
 

 
 Asian Brown Flycatcher 
 

 
 Oriental Magpie-Robin 
 

 
 Booted Eagle 
 

 
 Common Tailorbird 
 

 
 Malabar Barbet 
 

 
 sunbird sp. 
 

 
 Tawny-bellied Babbler 
 

 
 Pheasant-tailed Jacana 
 

 
 cormorant sp. 
 

 
 Lesser Whitethroat 
 

 
 Rock Pigeon 
 

 
 Temminck's Stint 
 

 
 Gray Heron 
 

 
 Asian Emerald Dove 
 

 
 Black Eagle 
 

 
 Yellow-footed Pigeon 
 

 
 Gray-headed Swamphen 
 

 
 White-cheeked Barbet 
 

 
 Paddyfield Pipit 
 

 
 Asian Fairy-bluebird 
 

 
 Little Swift 
 

 
 Common Snipe 
 

 
 Western Yellow Wagtail 
 

 
 Yellow-eyed Babbler 
 

 
 Asian Koel 
 

 
 White-browed Fantail 
 

 
 Crested Serpent-Eagle 
 

 
 Common Redshank 
 

 
 Laughing Dove 
 

 
 Indian Spot-billed Duck 
 

 
 Blyth's Reed-Warbler 
 

 
 White-browed Bulbul 
 

 
 Ashy-crowned Sparrow-Lark 
 

 
 Black Redstart 
 

 
 Rufous Babbler 
 

 
 Jerdon's Leafbird 
 

 
 Red-vented Bulbul 
 

 
 Green-winged Teal 
 

 
 Green Sandpiper 
 

 
 Indian Cormorant 
 

 
 Rosy Starling 
 

 
 Pied Bushchat 
 

 
 Black-tailed Godwit 
 

 
 Indian Pond-Heron 
 

 
 Brown Shrike 
 

 
 Eurasian Hoopoe 
 

 
 Hume's Warbler 
 

 
 Indian Scimitar-Babbler 
 

 
 Golden-fronted Leafbird 
 

 
 crow sp. 
 

 
 Plum-headed Parakeet 
 

 
 White-eyed Buzzard 
 

 
 Crimson-backed Sunbird 
 

 
 Lesser Whistling-Duck 
 

 
 Nilgiri Flowerpecker 
 

 
 Little Spiderhunter 
 

 
 Black-rumped Flameback 
 

 
 Malabar Starling 
 

 
 Barn Swallow 
 

 
 Gray-breasted Prinia 
 

 
 Eurasian Moorhen 
 

 
 Cotton Pygmy-Goose 
 

 
 Shikra 
 

 
 Taiga Flycatcher 
 

 
 Purple Sunbird 
 

 
 Black-headed Cuckooshrike 
 

 
 Chestnut-tailed Starling 
 

 
 Spotted Owlet 
 

 
 Indian Pitta 
 

 
 Eurasian Collared-Dove 
 

 
 Ashy Prinia 
 

 
 Marsh Sandpiper 
 

 
 Brahminy Starling 
 

 
 Clamorous Reed-Warbler 
 

 
 Black-shouldered Kite 
 

 
 River Tern 
 

 
 Brahminy Kite 
 

 
 Gray Francolin 
 

 
 Brown-headed Gull 
 

 
 Chestnut-shouldered Petronia 
 

 
 Greater Flameback 
 

 
 Common Hawk-Cuckoo 
 

 
 Gray-hooded Warbler 
 

 
 Oriental Honey-buzzard 
 

 
 Common Sandpiper 
 

 
 Indian Swiftlet 
 

 
 Red-rumped Swallow 
 

 
 Scaly-breasted Munia 
 

 
 Black-naped Monarch 
 

 
 Blue-tailed Bee-eater 
 

 
 Cine

To the right of the picture are himalayan birds, and to the left are those from Malabar. The lower part of the  main cluster is mainly water birds.

As geographical factors dominate the picture, it is worth separating these from others such as habitats. To do this, we take a variant of co-occurence where we take the ratio of the probability of a pair of species occuring together to what the  probability would be if their occurence were independent _conditioned on the geographic distribution_.

In [25]:
val birdLocFreqF = data / "bird-location-freqs.tsv"
val locFreqF = data / "location-freqs.tsv"

val locationFreqs = read.lines(locFreqF) map(_.split("\t")) map {case Array(l, n) => (l, n.toInt)} toMap

birdLocFreqF: data.ThisType = /home/gadgil/code/ProvingGround/data/bird-location-freqs.tsv
locFreqF: data.ThisType = /home/gadgil/code/ProvingGround/data/location-freqs.tsv
locationFreqs: Map[String, Int] = Map(
  "IN-DL" -> 357,
  "IN-MH" -> 1171,
  "IN-AP" -> 249,
  "IN-LD" -> 2,
  "IN-DD" -> 4,
  "IN-CH" -> 15,
  "IN-MP" -> 139,
  "IN-PY" -> 65,
  "IN-TR" -> 14,
  "IN-OR" -> 83,
  "IN-UL" -> 1354,
  "IN-GA" -> 1396,
  "IN-BR" -> 217,
  "IN-ML" -> 53,
  "IN-GJ" -> 463,
  "IN-DN" -> 1,
  "IN-TN" -> 4127,
  "IN-JK" -> 227,
  "IN-KL" -> 11632,
...

In [26]:
val birdLocFreqs = read.lines(birdLocFreqF) map(_.split("\t")) map {case Array(b, l, n) => ((b, l), n.toInt)} toMap

birdLocFreqs: Map[(String, String), Int] = Map(
  ("Motacilla flava", "IN-BR") -> 2,
  ("Parulidae sp.", "IN-KL") -> 13,
  ("Ficedula hodgsoni", "IN-WB") -> 16,
  ("Mirafra erythroptera", "IN-BR") -> 8,
  ("Prunella immaculata", "IN-WB") -> 23,
  ("Lophura leucomelanos", "IN-NL") -> 29,
  ("Gallirallus striatus", "IN-GJ") -> 1,
  ("Bubulcus ibis", "IN-UP") -> 1552,
  ("Anthus cervinus", "IN-HR") -> 2,
  ("Anas querquedula/crecca", "IN-TN") -> 1,
  ("Mycteria leucocephala", "IN-AS") -> 12,
  ("Circus aeruginosus", "IN-BR") -> 6,
  ("Aegithalos concinnus", "IN-NL") -> 33,
  ("Oriolus sp.", "IN-UL") -> 11,
  ("Muscicapa sp.", "IN-ML") -> 4,
  ("Psittacula eupatria", "IN-OR") -> 73,
  ("Corvus sp. (crow sp.)", "IN-AR") -> 4,
  ("Dicaeum agile", "IN-JH") -> 4,
  ("Nyctyornis athertoni", "IN-MH") -> 5,
...

In [27]:
val places = locationFreqs.keys.toVector

places: Vector[String] = Vector(
  "IN-DL",
  "IN-MH",
  "IN-AP",
  "IN-LD",
  "IN-DD",
  "IN-CH",
  "IN-MP",
  "IN-PY",
  "IN-TR",
  "IN-OR",
  "IN-UL",
  "IN-GA",
  "IN-BR",
  "IN-ML",
  "IN-GJ",
  "IN-DN",
  "IN-TN",
  "IN-JK",
  "IN-KL",
...

In [28]:
def birdLocNum(bird: String, loc: String) = birdLocFreqs.getOrElse((bird, loc), 0).toDouble

defined function birdLocNum

In [29]:
def pairInLoc(x: String, y: String, loc: String) = birdLocNum(x, loc) * birdLocNum(y, loc) / locationFreqs(loc)

defined function pairInLoc

In [30]:
def seenSameLoc(x: String, y: String) = {places map (pairInLoc(x, y, _))}.sum

defined function seenSameLoc

In [31]:
def localCoOccurence(x: String, y: String) = bothSeen(x, y) / seenSameLoc(x, y)

defined function localCoOccurence

In [32]:
val byLocalCoOcc = {for{ (x, y) <- topPairs if seenSameLoc(x, y) > 0} yield 
                (x, y, localCoOccurence(x, y))} sortBy ((t) => -t._3)

byLocalCoOcc: Vector[(String, String, Double)] = Vector(
  ("Gracula indica", "Rhipidura albicollis", 465.3333333333333),
  ("Dendrocitta leucogastra", "Rhipidura albicollis", 465.3333333333333),
  ("Rhipidura albicollis", "Gracula indica", 465.3333333333333),
  ("Rhipidura albicollis", "Dendrocitta leucogastra", 465.3333333333333),
  ("Mirafra affinis", "Saxicola ferreus", 390.3333333333333),
  ("Saxicola ferreus", "Mirafra affinis", 390.3333333333333),
  ("Psilopogon malabaricus", "Saxicola ferreus", 292.75),
  ("Saxicola ferreus", "Psilopogon malabaricus", 292.75),
  ("Turdoides affinis", "Saxicola ferreus", 234.20000000000002),
  ("Saxicola ferreus", "Turdoides affinis", 234.20000000000002),
  ("Aerodramus unicolor", "Saxicola ferreus", 167.2857142857143),
  ("Saxicola ferreus", "Aerodramus unicolor", 167.2857142857143),
  ("Gracula indica", "Acridotheres ginginianus", 155.11111111111111),
  ("Acridotheres ginginianus", "Gracula indica", 155.11111111111111),
  ("Acridotheres gingin

In [33]:
show(byLocalCoOcc map {case (x, y, n) => (commonNames(x), commonNames(y), n)} take 1000)

Vector(
  ("Southern Hill Myna", "White-throated Fantail", 465.3333333333333),
  ("White-bellied Treepie", "White-throated Fantail", 465.3333333333333),
  ("White-throated Fantail", "Southern Hill Myna", 465.3333333333333),
  ("White-throated Fantail", "White-bellied Treepie", 465.3333333333333),
  ("Jerdon's Bushlark", "Gray Bushchat", 390.3333333333333),
  ("Gray Bushchat", "Jerdon's Bushlark", 390.3333333333333),
  ("Malabar Barbet", "Gray Bushchat", 292.75),
  ("Gray Bushchat", "Malabar Barbet", 292.75),
  ("Yellow-billed Babbler", "Gray Bushchat", 234.20000000000002),
  ("Gray Bushchat", "Yellow-billed Babbler", 234.20000000000002),
  ("Indian Swiftlet", "Gray Bushchat", 167.2857142857143),
  ("Gray Bushchat", "Indian Swiftlet", 167.2857142857143),
  ("Southern Hill Myna", "Bank Myna", 155.11111111111111),
  ("Bank Myna", "Southern Hill Myna", 155.11111111111111),
  ("Bank Myna", "White-bellied Treepie", 155.11111111111111),
  ("White-bellied Treepie", "Bank Myna", 155.11111111111

In [34]:
val birdLocEdges = for ((x, y) <- topPairs) yield Edge(Node(x, commonNames(x)), Node(y, commonNames(y)), localCoOccurence(x, y))

birdLocEdges: Vector[Edge] = Vector(
  Edge(
    Node(
      "Corvus splendens",
      "House Crow",
      1.0,
      0,
      List(),
      List(),
      NodeState(
        Vector2D(0.4839669666914944, 0.4848822396937428),
        Vector2D(0.0, 0.0),
        Vector2D(0.0, 0.0)
      )
    ),
    Node(
      "Acridotheres tristis",
      "Common Myna",
      1.0,
      0,
...

In [35]:
val birdLocGraph = new SpringGraph(birdNodes, birdLocEdges)

birdLocGraph: SpringGraph = at.ait.dme.forcelayout.SpringGraph@5d4dab43

In [36]:
birdLocGraph.doLayout(maxIterations = 3000)

In [37]:
val birdLocTriples = birdLocGraph.nodes.toVector map ((n) => (n.state.pos.x, n.state.pos.y, n.label))

birdLocTriples: Vector[(Double, Double, String)] = Vector(
  (1208.5794247404278, 753.6699889774096, "Alexandrine Parakeet"),
  (-458.16670543294015, -536.2105051943871, "Large-billed Crow"),
  (-757.1440727866661, 1287.6958046326067, "Great Egret"),
  (-1431.0742321824405, 3892.769002085436, "Painted Stork"),
  (473.3690936750347, -1639.7080922546318, "Oriental White-eye"),
  (993.5467289042905, -2758.681601265033, "White-bellied Treepie"),
  (-299.3997492786954, 1587.1546889558756, "Baya Weaver"),
  (-24.274335130624408, 125.90195005396666, "Ashy Woodswallow"),
  (684.8407269152327, -1307.350226579547, "Tickell's Blue-Flycatcher"),
  (-1549.4886415384967, 3502.5165398502036, "Eurasian Spoonbill"),
  (945.6370779991622, -1900.9132753009599, "Thick-billed Flowerpecker"),
  (696.2408089755646, -1709.6681534442457, "Lesser Yellownape"),
  (585.6164010380959, -1921.1604531798112, "Verditer Flycatcher"),
  (578.5648280893312, -1419.9347540654426, "Brown-capped Woodpecker"),
  (-1195.819896

In [38]:
val birdLocPlot = scatterPlot(birdLocTriples, width = 1200, height = 500, r = 2)

birdLocPlot: String = """

      <div>
      
    
    <style>
      .labl {
        display: none;
      }
      .labelled:hover + .labl {
        display: inline;
      }
    </style>
    

    <svg version="1.1"
   baseProfile="full"
   width="1200" height="500"
   xmlns="http://www.w3.org/2000/svg">

...

In [39]:
display.html(birdLocPlot)

Alexandrine Parakeet 
 

 
 Large-billed Crow 
 

 
 Great Egret 
 

 
 Painted Stork 
 

 
 Oriental White-eye 
 

 
 White-bellied Treepie 
 

 
 Baya Weaver 
 

 
 Ashy Woodswallow 
 

 
 Tickell's Blue-Flycatcher 
 

 
 Eurasian Spoonbill 
 

 
 Thick-billed Flowerpecker 
 

 
 Lesser Yellownape 
 

 
 Verditer Flycatcher 
 

 
 Brown-capped Woodpecker 
 

 
 Bank Myna 
 

 
 Eurasian Marsh-Harrier 
 

 
 Chestnut-headed Bee-eater 
 

 
 Blue-faced Malkoha 
 

 
 Red-breasted Flycatcher 
 

 
 Pied Kingfisher 
 

 
 Oriental Turtle-Dove 
 

 
 Tricolored Munia 
 

 
 Malabar Gray Hornbill 
 

 
 Short-toed Snake-Eagle 
 

 
 Southern Hill Myna 
 

 
 Himalayan Bulbul 
 

 
 Common Iora 
 

 
 Greenish Warbler 
 

 
 Bay-backed Shrike 
 

 
 Greater Coucal 
 

 
 Gray Wagtail 
 

 
 Indian Gray Hornbill 
 

 
 Red Spurfowl 
 

 
 Stork-billed Kingfisher 
 

 
 Bronze-winged Jacana 
 

 
 Orange Minivet 
 

 
 Red-wattled Lapwing 
 

 
 Pale-billed Flowerpecker 
 

 
 Jungle Prinia 
 

 
 Eurasian Kestrel 
 

 
 Asian Openbill 
 

 
 Indian Golden Oriole 
 

 
 Red-whiskered Bulbul 
 

 
 White Wagtail 
 

 
 Booted Warbler 
 

 
 Kentish Plover 
 

 
 Gray-fronted Green-Pigeon 
 

 
 Jungle Owlet 
 

 
 Long-tailed Shrike 
 

 
 House Sparrow 
 

 
 white egret sp. 
 

 
 Rufous Woodpecker 
 

 
 Hair-crested Drongo 
 

 
 Malabar Woodshrike 
 

 
 Ashy Drongo 
 

 
 Plain Prinia 
 

 
 Indian Silverbill 
 

 
 Siberian Stonechat 
 

 
 Green/Greenish Warbler 
 

 
 Rufous Treepie 
 

 
 Brown-headed Barbet 
 

 
 Gray Junglefowl 
 

 
 Little Stint 
 

 
 Osprey 
 

 
 Pied Cuckoo 
 

 
 Little Ringed Plover 
 

 
 Garganey 
 

 
 Coppersmith Barbet 
 

 
 Black Bulbul 
 

 
 Spot-billed Pelican 
 

 
 Asian Brown Flycatcher 
 

 
 Oriental Magpie-Robin 
 

 
 Booted Eagle 
 

 
 Common Tailorbird 
 

 
 Malabar Barbet 
 

 
 sunbird sp. 
 

 
 Tawny-bellied Babbler 
 

 
 Pheasant-tailed Jacana 
 

 
 cormorant sp. 
 

 
 Lesser Whitethroat 
 

 
 Rock Pigeon 
 

 
 Temminck's Stint 
 

 
 Gray Heron 
 

 
 Asian Emerald Dove 
 

 
 Black Eagle 
 

 
 Yellow-footed Pigeon 
 

 
 Gray-headed Swamphen 
 

 
 White-cheeked Barbet 
 

 
 Paddyfield Pipit 
 

 
 Asian Fairy-bluebird 
 

 
 Little Swift 
 

 
 Common Snipe 
 

 
 Western Yellow Wagtail 
 

 
 Yellow-eyed Babbler 
 

 
 Asian Koel 
 

 
 White-browed Fantail 
 

 
 Crested Serpent-Eagle 
 

 
 Common Redshank 
 

 
 Laughing Dove 
 

 
 Indian Spot-billed Duck 
 

 
 Blyth's Reed-Warbler 
 

 
 White-browed Bulbul 
 

 
 Ashy-crowned Sparrow-Lark 
 

 
 Black Redstart 
 

 
 Rufous Babbler 
 

 
 Jerdon's Leafbird 
 

 
 Red-vented Bulbul 
 

 
 Green-winged Teal 
 

 
 Green Sandpiper 
 

 
 Indian Cormorant 
 

 
 Rosy Starling 
 

 
 Pied Bushchat 
 

 
 Black-tailed Godwit 
 

 
 Indian Pond-Heron 
 

 
 Brown Shrike 
 

 
 Eurasian Hoopoe 
 

 
 Hume's Warbler 
 

 
 Indian Scimitar-Babbler 
 

 
 Golden-fronted Leafbird 
 

 
 crow sp. 
 

 
 Plum-headed Parakeet 
 

 
 White-eyed Buzzard 
 

 
 Crimson-backed Sunbird 
 

 
 Lesser Whistling-Duck 
 

 
 Nilgiri Flowerpecker 
 

 
 Little Spiderhunter 
 

 
 Black-rumped Flameback 
 

 
 Malabar Starling 
 

 
 Barn Swallow 
 

 
 Gray-breasted Prinia 
 

 
 Eurasian Moorhen 
 

 
 Cotton Pygmy-Goose 
 

 
 Shikra 
 

 
 Taiga Flycatcher 
 

 
 Purple Sunbird 
 

 
 Black-headed Cuckooshrike 
 

 
 Chestnut-tailed Starling 
 

 
 Spotted Owlet 
 

 
 Indian Pitta 
 

 
 Eurasian Collared-Dove 
 

 
 Ashy Prinia 
 

 
 Marsh Sandpiper 
 

 
 Brahminy Starling 
 

 
 Clamorous Reed-Warbler 
 

 
 Black-shouldered Kite 
 

 
 River Tern 
 

 
 Brahminy Kite 
 

 
 Gray Francolin 
 

 
 Brown-headed Gull 
 

 
 Chestnut-shouldered Petronia 
 

 
 Greater Flameback 
 

 
 Common Hawk-Cuckoo 
 

 
 Gray-hooded Warbler 
 

 
 Oriental Honey-buzzard 
 

 
 Common Sandpiper 
 

 
 Indian Swiftlet 
 

 
 Red-rumped Swallow 
 

 
 Scaly-breasted Munia 
 

 
 Black-naped Monarch 
 

 
 Blue-tailed Bee-eater 
 

 
 Cine

Once the geography has been separated out, one can see that habitats dominate. On the left end are clearly waterbirds, on the right it appears that we have birds that are in forested areas.

### Concluding remarks:

Given the limitations of the present e-bird data, it is clear that we cannot go much beyond this - indeed only the geographical regions and habitats that have the greatest impact on birds are visible. Nevertheless we speculate on analysis that can be done with more data of the same nature.

* We can look for species A and B such that
    * A and B are close in the force-directed graph, but
    * A and B do not co-occur as much as expected given their proximity.
 
 such species are likely to be occupying the same niche.
 
 * As mentioned in the introduction, one can naturally partition into _geographical regions_, _habitats_ and even _micro-habitats_, with geographical region clustering based on geodata as well as clustering of co-occurences.